In [1]:
# install libraries
# !pip install gspread
# !pip install matplotlib
# !pip install oauth2client

In [2]:
# pip install google-api-python-client google-auth-httplib2 google-auth-oauthlib

In [9]:
# # import libraries
# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt
# import gspread
# from oauth2client.service_account import ServiceAccountCredentials
# from oauth2client.client import GoogleCredentials

In [3]:
# # https://youtu.be/3wC-SCdJK2c?t=507
# import os
# from google.auth.transport.requests import Request
# from google.oauth2.credentials import Credentials
# from google_auth_oauthlib.flow import InstalledAppFlow
# from googleapiclient.discovery import build
# from googleapiclient.errors import HttpError

In [ ]:
# SCOPES = ["https://googleapis.com/auth/spreadsheets.readonly"]
# SPREADSHEET_ID = "1b9yRWonjluWSq9TxpLf9m-vecrnZtN_BzxptGwhuteM/edit?resourcekey#gid=1191109888"

# Ignore everything above. Tried to connect Python with Google Sheets with Oauth. Will continue later.
# Now I'm downloading the dataset and deal with it locally.

In [3]:
# Fuzz matching packages
# !pip install thefuzz
from thefuzz import fuzz
from thefuzz import process

In [4]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [5]:
Survey_dir = 'C:/Users/gaoji/Dropbox/副业/It takes a village/Student Information Survey 23.10.27.xlsx' # most up-to-date version
StudentList_dir = 'C:/Users/gaoji/Dropbox/副业/It takes a village/Student List.xlsx'
LibraryList_dir = 'C:/Users/gaoji/Dropbox/副业/It takes a village/Student List.xlsx'

In [10]:
SurveyDF = pd.read_excel(Survey_dir, sheet_name = "Form Responses 1")
StudentListDF = pd.read_excel(StudentList_dir, sheet_name = "Active Student List")
LibraryListDF = pd.read_excel(LibraryList_dir, sheet_name = "Overview")

In [11]:
# List of libraries of interest
libs_of_interest = ['Adeiso', 
                    'Sankubanase', 
                    'Ahero', 
                    'Maragoli', 
                    'Biwi', 
                    'Dzaleka', 
                    'Kadzakalowa', 
                    'Namalimwe', 
                    'Charity Centre', 
                    'High Tech',
                    'Jollyland',
                    'Mukono',
                    'Little Flower', 
                    'Cuchapa', 
                    'Darwin']

condition = LibraryListDF['Library'].isin(libs_of_interest)
Lib_filtered = LibraryListDF[condition]
Lib_filtered = Lib_filtered[['Library', 'Country', 'count', 'Librarains', 'Sent']]

In [12]:
# Generate new columns in StudentListDF to store survey-related info
StudentListDF['Student Survey Completed'] = 'No'
StudentListDF["Survey Name Error"] = ''
StudentListDF["Survey Completion Time"] = ''

# generate my own last name first name list, compare it to survey name
StudentListDF["last_first reverse"] = StudentListDF['last_name'] + ' ' + StudentListDF['first_name']

In [16]:
# Fuzzy matching
MatchingNameList_StuList = []
MatchingNameList_Survey = []
for i in range(len(StudentListDF["user_name"])):
    for j in range(len(SurveyDF["I-1. What's your full name?"])):
        
        # compare survey name with user name: perfect match
        if fuzz.ratio(StudentListDF["user_name"][i],SurveyDF["I-1. What's your full name?"][j]) == 100:
            StudentListDF.loc[i, 'Student Survey Completed'] =  "Completed"
            StudentListDF.loc[i, 'Survey Completion Time'] = SurveyDF["Timestamp"][j]
        
        # compare survey name with user name: not perfect match
        elif (fuzz.ratio(StudentListDF["user_name"][i],SurveyDF["I-1. What's your full name?"][j]) >= 80) and (fuzz.ratio(StudentListDF["user_name"][i],SurveyDF["I-1. What's your full name?"][j]) < 100):
            StudentListDF.loc[i, 'Student Survey Completed'] =  "Need Verification"
            StudentListDF.loc[i, "Survey Name Error"] = SurveyDF["I-1. What's your full name?"][j]
            StudentListDF.loc[i, 'Survey Completion Time'] = SurveyDF["Timestamp"][j]
        
        # compare survey name with last_first reverse
        elif fuzz.ratio(StudentListDF["last_first reverse"][i],SurveyDF["I-1. What's your full name?"][j]) >= 90:
            StudentListDF.loc[i, 'Student Survey Completed'] =  "Need Verification"
            StudentListDF.loc[i, "Survey Name Error"] = SurveyDF["I-1. What's your full name?"][j]
            StudentListDF.loc[i, 'Survey Completion Time'] = SurveyDF["Timestamp"][j]
            
SurveyCompletion_df = StudentListDF[['user_name', 'library_name', 'first_name','last_name', 'Drop-out', 'Student Survey Completed', 'Survey Name Error', 'Survey Completion Time']]
SurveyCompletion_df.to_excel('SurveyCompletion.xlsx', index = False)
# print(len(MatchingNameList_Survey))